In [199]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Cost Supply Network Optimization

### Problem Statement
###### Come up with a best cost supply network (Which supplier which location should service which plants and what qty). It must optimize all constraints, satisfy the full demand at our locations.


**Optimization Function**: Delivered cost = Base Cost + Freight Cost + Any taxes/duties
##### Some additional Points to be taken care of
* If BC Supply <19000 tons, then we incur a penalty of $125/MT
* Supplies to China from outside of South East Asia, incurs an import duty of 15%
* Suppliers to "AP - China" from outside of South East Asia, incurs an import duty of 18%
* If EU is supplied from Asia based suppliers, there is an import duty of 8%.


#### The first step is to open all data sources

##### base_cost

In [200]:
base_cost = pd.read_csv('../data/raw/base_cost.csv', sep=';', decimal=',', encoding='UTF-8')

In [201]:
base_cost

,Location,Cost ($/MT)
0,BC San Pedro,3496
1,BC Malaysia,3942
2,BC Pasir Gudang,3942
3,Olam Singapore,3836
4,Olam Holland,3278


This dataframe will be converted to a numpy array and transposed to ease the calculations

In [202]:
base_cost=np.array((base_cost.values[:,1]),ndmin=2)

In [203]:
base_cost

array([[3496, 3942, 3942, 3836, 3278]], dtype=object)

##### demand

In [204]:
demand=pd.read_csv('../data/raw/demand.csv', sep=';', decimal=',', encoding='UTF-8')

In [205]:
demand

,destiny,quantity
0,East Coast BB,4080
1,East Coast SB,1121
2,West Coast BB,624
3,Salinas BB,5585
4,EU,4607
5,EEMEA,0
6,LA,1703
7,AP - China,3571
8,China,5345


##### freight_matrix

In [206]:
freight_matrix=pd.read_csv('../data/raw/freight_matrix.csv', sep=';', decimal=',', encoding='UTF-8')

In [207]:
freight_matrix

,Unnamed: 0,BC San Pedro,BC Malaysia,BC Pasir Gudang,ADM Singapore,ADM Amsterdam
0,East Coast BB,111,240,240,140,120
1,East Coast SB,111,240,240,140,120
2,West Coast BB,511,240,240,200,120
3,Salinas BB,620,445,445,350,300
4,EU,75,150,150,150,50
5,EEMEA,163,41,41,41,62
6,LA,120,110,110,250,80
7,AP - China,250,60,60,60,250
8,China,300,60,60,60,350


In [208]:
freight_matrix.rename(columns={'Unnamed: 0':''}, inplace=True)
freight_matrix.set_index('',inplace=True)

In [209]:
freight_matrix

,BC San Pedro,BC Malaysia,BC Pasir Gudang,ADM Singapore,ADM Amsterdam
,,,,,
East Coast BB,111,240,240,140,120
East Coast SB,111,240,240,140,120
West Coast BB,511,240,240,200,120
Salinas BB,620,445,445,350,300
EU,75,150,150,150,50
EEMEA,163,41,41,41,62
LA,120,110,110,250,80
AP - China,250,60,60,60,250
China,300,60,60,60,350


##### supply_location_capacity

In [210]:
supply_location_capacity=pd.read_csv('../data/raw/supply_location_capacity.csv', sep=';', decimal=',', encoding='UTF-8')

In [211]:
supply_location_capacity

,Supplier,Location,Max Capacity
0,BC,BC San Pedro,12480
1,BC,BC Malaysia,7200
2,BC,BC Pasir Gudang,7200
3,Olam,Olam Singapore,7500
4,Olam,Olam Holland,6000


##### quantity
This matrix will be started with 0 but the zeros will be optimized 

In [212]:
quantity=np.zeros((9,5))
quantity

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

### Optimization Function

The main objective of this code is to minimize the below function

In [213]:
def objective_calculator(base,freight,taxes):
    return base+freight+taxes

### Support functions

Let´s start defining how the freight_cost is calculated

In [340]:
def matrix_freight_cost_calculator(quantity, freight_matrix):
    return np.reshape(quantity, (-1, 5))*freight_matrix

In [361]:
def total_freight_cost_calculator(matrix):
    return np.sum(matrix.values)

In [342]:
def matrix_base_cost_calculator(quantity,base_cost):
    return np.reshape(quantity, (-1, 5))*base_cost

In [343]:
def total_base_cost_calculator(matrix):      
    return np.sum(matrix)

### Conditions

If BC Supply <19000 tons, then we incur a penalty of $125/MT

In [346]:
def bc_supply_penalty(quantity):
    # BC supplyers are the first 3 columns of the matrix
    # We will sum them and apply the above rule if the result is < 19000
    bc_quantity= np.sum(np.reshape(quantity, (-1, 5))[:,:3])
    return bc_quantity*125 if bc_quantity <19000 else 0

Supplies to China from outside of South East Asia, incurs an import duty of 15%

In [246]:
def china_tax(calculated_freight_matrix,base_cost_matrix):
    # The only supplier outside South East Asia is ADM Amsterdam
    # The tax will be calculated over the freight cost + base cost for that supplier to Chine
    # The position of this supplyer to China on the calculated_freigh_matrix and base_cost_matrix is 8,4 (9th row, 5th column)
    return ((calculated_freight_matrix.values[8,4])+(base_cost_matrix[8,4]))*0.15

Suppliers to "AP - China" from outside of South East Asia, incurs an import duty of 18%

In [248]:
def ap_china_tax(calculated_freight_matrix,base_cost_matrix):
    # The only supplier outside South East Asia is ADM Amsterdam
    # The tax will be calculated over the freight cost + base cost for that supplier to Chine
    # The position of this supplyer to AP-China on the calculated_freigh_matrix and base_cost_matrix is 7,4 (8th row, 5th column)
    return (calculated_freight_matrix.values[7,4]+base_cost_matrix[7,4])*0.18

If EU is supplied from Asia based suppliers, there is an import duty of 8%.

In [249]:
def eu_asia_tax(calculated_freight_matrix,base_cost_matrix):
    # EU is in row 4 of the matrix (5th line)
    # Asia based suppliers are columns 0 to 3 (4 columns)
    return np.sum((calculated_freight_matrix.values[4,:3]+base_cost_matrix[4,:3]))*0.08

### Time to sum all taxes

In [251]:
def all_taxes_calculator(quantity,calculated_freight_matrix,base_cost_matrix):
    results=[]
    results.append(bc_supply_penalty(quantity))
    results.append(china_tax(calculated_freight_matrix,base_cost_matrix))
    results.append(ap_china_tax(calculated_freight_matrix,base_cost_matrix))
    results.append(eu_asia_tax(calculated_freight_matrix,base_cost_matrix))
    return np.sum(results)

In [252]:
taxes = all_taxes_calculator(quantity,
                             matrix_freight_cost_calculator(quantity, freight_matrix),
                             matrix_base_cost_calculator(quantity,base_cost))

In [253]:
base = total_base_cost_calculator(matrix_base_cost_calculator(quantity,base_cost))

In [254]:
freight = total_freight_cost_calculator(matrix_freight_cost_calculator(quantity, freight_matrix))

### Constraints

#### Demand Constraints

In [308]:
quantity

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [321]:
def constraint1(quantity):
    q = np.reshape(quantity, (-1, 5))    
    return q.sum(axis=1)[0]-4080

In [322]:
def constraint2(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[1]-1121

In [323]:
def constraint3(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[2]-624

In [324]:
def constraint4(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[3]-5585

In [325]:
def constraint5(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[4]-4607

In [326]:
def constraint6(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[5]-0

In [327]:
def constraint7(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[6]-1703

In [328]:
def constraint8(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[7]-3571

In [329]:
def constraint9(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return q.sum(axis=1)[8]-5345

#### Capacity Constraints

In [330]:
def constraint10(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 12480-q.sum(axis=0)[0]

In [331]:
def constraint11(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7200-q.sum(axis=0)[1]

In [332]:
def constraint12(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7200-q.sum(axis=0)[2]

In [333]:
def constraint13(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 7500-q.sum(axis=0)[3]

In [334]:
def constraint14(quantity):
    q = np.reshape(quantity, (-1, 5))   
    return 6000-q.sum(axis=0)[4]

In [426]:
# def constraint15(quantity):
#     validator = True
#     for item in quantity: 
#         validator=validator and max([item-int(item)])==0
#     return 0 if validator else 100

In [427]:
con1 = {'type':'ineq', 'fun':constraint1}
con2 = {'type':'ineq', 'fun':constraint2}
con3 = {'type':'ineq', 'fun':constraint3}
con4 = {'type':'ineq', 'fun':constraint4}
con5 = {'type':'ineq', 'fun':constraint5}
con6 = {'type':'ineq', 'fun':constraint6}
con7 = {'type':'ineq', 'fun':constraint7}
con8 = {'type':'ineq', 'fun':constraint8}
con9 = {'type':'ineq', 'fun':constraint9}
con10 = {'type':'ineq', 'fun':constraint10}
con11 = {'type':'ineq', 'fun':constraint11}
con12 = {'type':'ineq', 'fun':constraint12}
con13 = {'type':'ineq', 'fun':constraint13}
con14 = {'type':'ineq', 'fun':constraint14}
# con15 = {'type':'eq', 'fun':constraint15}

In [428]:
cons=[con1,con2,con3,con4,con5,con6,con7,
     con8,con9,con10,con11,con12,con13,con14]

In [429]:
b=(0., None)
bnds=(b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b,
      b,b,b,b,b)

In [430]:
def objective_calculator(quantity,freight_matrix,base_cost):
    taxes = all_taxes_calculator(quantity,
                             matrix_freight_cost_calculator(quantity, freight_matrix),
                             matrix_base_cost_calculator(quantity,base_cost))
    base = total_base_cost_calculator(matrix_base_cost_calculator(quantity,base_cost))
    freight = total_freight_cost_calculator(matrix_freight_cost_calculator(quantity, freight_matrix))
    return base+freight+taxes

In [431]:
minimize(objective_calculator,quantity,method='SLSQP', constraints=cons ,bounds=bnds,args=(freight_matrix,base_cost))

     fun: 100993832.06330536
     jac: array([3732., 4307., 4307., 3976., 3398., 3732., 4307., 4307., 3976.,
       3398., 4132., 4307., 4307., 4036., 3398., 4241., 4512., 4512.,
       4186., 3578., 3981., 4544., 4544., 3986., 3328., 3784., 4108.,
       4108., 3877., 3340., 3741., 4177., 4177., 4086., 3358., 3871.,
       4127., 4127., 3896., 4163., 3921., 4127., 4127., 3896., 4172.])
 message: 'Positive directional derivative for linesearch'
    nfev: 1271
     nit: 31
    njev: 27
  status: 8
 success: False
       x: array([4079.99891   ,    0.        ,    0.        ,    0.        ,
          0.        , 1120.99891   ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,  623.99900755,    0.        ,
          0.        ,    0.        , 4815.99877744,  768.99895494,
          0.        ,    0.        ,    0.        ,    0.        ,
       4606.99902799,    0.        ,    0.        ,    0.        ,
   

## Scipy SLSQP doesn´t have an integer optimizator, so we will try a different one

In [450]:
import pulp

In [451]:
# create the LP object, set up as a minimization problem
prob = pulp.LpProblem('Cost Supply Network', pulp.LpMinimize)

In [452]:
supplyer=['BC San Pedro','BC Malaysia','BC Pasir Gudang','ADM Singapore','ADM Amsterdam']
demand=['East Coast BB','East Coast SB','West Coast BB','Salinas BB','EU','EEMEA','LA','AP - China','China']

In [467]:
vars=[]
for s in supplyer:
    s=s.replace(' ','_')
    for d in demand:
        d=d.replace(' ','_')        
        d=d.replace('-','')
        d=d.replace('__','_')
        vars.append(d)

In [466]:
BC_San_Pedro_to_East_Coast_BB=pulp.LpVariable('BC_San_Pedro_to_East_Coast_BB', lowBound=0, cat='Integer')
BC_San_Pedro_to_East_Coast_SB=pulp.LpVariable('BC_San_Pedro_to_East_Coast_SB', lowBound=0, cat='Integer')
BC_San_Pedro_to_West_Coast_BB=pulp.LpVariable('BC_San_Pedro_to_West_Coast_BB', lowBound=0, cat='Integer')
BC_San_Pedro_to_Salinas_BB=pulp.LpVariable('BC_San_Pedro_to_Salinas_BB', lowBound=0, cat='Integer')
BC_San_Pedro_to_EU=pulp.LpVariable('BC_San_Pedro_to_EU', lowBound=0, cat='Integer')
BC_San_Pedro_to_EEMEA=pulp.LpVariable('BC_San_Pedro_to_EEMEA', lowBound=0, cat='Integer')
BC_San_Pedro_to_LA=pulp.LpVariable('BC_San_Pedro_to_LA', lowBound=0, cat='Integer')
BC_San_Pedro_to_AP_China=pulp.LpVariable('BC_San_Pedro_to_AP_China', lowBound=0, cat='Integer')
BC_San_Pedro_to_China=pulp.LpVariable('BC_San_Pedro_to_China', lowBound=0, cat='Integer')
BC_Malaysia_to_East_Coast_BB=pulp.LpVariable('BC_Malaysia_to_East_Coast_BB', lowBound=0, cat='Integer')
BC_Malaysia_to_East_Coast_SB=pulp.LpVariable('BC_Malaysia_to_East_Coast_SB', lowBound=0, cat='Integer')
BC_Malaysia_to_West_Coast_BB=pulp.LpVariable('BC_Malaysia_to_West_Coast_BB', lowBound=0, cat='Integer')
BC_Malaysia_to_Salinas_BB=pulp.LpVariable('BC_Malaysia_to_Salinas_BB', lowBound=0, cat='Integer')
BC_Malaysia_to_EU=pulp.LpVariable('BC_Malaysia_to_EU', lowBound=0, cat='Integer')
BC_Malaysia_to_EEMEA=pulp.LpVariable('BC_Malaysia_to_EEMEA', lowBound=0, cat='Integer')
BC_Malaysia_to_LA=pulp.LpVariable('BC_Malaysia_to_LA', lowBound=0, cat='Integer')
BC_Malaysia_to_AP_China=pulp.LpVariable('BC_Malaysia_to_AP_China', lowBound=0, cat='Integer')
BC_Malaysia_to_China=pulp.LpVariable('BC_Malaysia_to_China', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_East_Coast_BB=pulp.LpVariable('BC_Pasir_Gudang_to_East_Coast_BB', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_East_Coast_SB=pulp.LpVariable('BC_Pasir_Gudang_to_East_Coast_SB', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_West_Coast_BB=pulp.LpVariable('BC_Pasir_Gudang_to_West_Coast_BB', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_Salinas_BB=pulp.LpVariable('BC_Pasir_Gudang_to_Salinas_BB', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_EU=pulp.LpVariable('BC_Pasir_Gudang_to_EU', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_EEMEA=pulp.LpVariable('BC_Pasir_Gudang_to_EEMEA', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_LA=pulp.LpVariable('BC_Pasir_Gudang_to_LA', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_AP_China=pulp.LpVariable('BC_Pasir_Gudang_to_AP_China', lowBound=0, cat='Integer')
BC_Pasir_Gudang_to_China=pulp.LpVariable('BC_Pasir_Gudang_to_China', lowBound=0, cat='Integer')
ADM_Singapore_to_East_Coast_BB=pulp.LpVariable('ADM_Singapore_to_East_Coast_BB', lowBound=0, cat='Integer')
ADM_Singapore_to_East_Coast_SB=pulp.LpVariable('ADM_Singapore_to_East_Coast_SB', lowBound=0, cat='Integer')
ADM_Singapore_to_West_Coast_BB=pulp.LpVariable('ADM_Singapore_to_West_Coast_BB', lowBound=0, cat='Integer')
ADM_Singapore_to_Salinas_BB=pulp.LpVariable('ADM_Singapore_to_Salinas_BB', lowBound=0, cat='Integer')
ADM_Singapore_to_EU=pulp.LpVariable('ADM_Singapore_to_EU', lowBound=0, cat='Integer')
ADM_Singapore_to_EEMEA=pulp.LpVariable('ADM_Singapore_to_EEMEA', lowBound=0, cat='Integer')
ADM_Singapore_to_LA=pulp.LpVariable('ADM_Singapore_to_LA', lowBound=0, cat='Integer')
ADM_Singapore_to_AP_China=pulp.LpVariable('ADM_Singapore_to_AP_China', lowBound=0, cat='Integer')
ADM_Singapore_to_China=pulp.LpVariable('ADM_Singapore_to_China', lowBound=0, cat='Integer')
ADM_Amsterdam_to_East_Coast_BB=pulp.LpVariable('ADM_Amsterdam_to_East_Coast_BB', lowBound=0, cat='Integer')
ADM_Amsterdam_to_East_Coast_SB=pulp.LpVariable('ADM_Amsterdam_to_East_Coast_SB', lowBound=0, cat='Integer')
ADM_Amsterdam_to_West_Coast_BB=pulp.LpVariable('ADM_Amsterdam_to_West_Coast_BB', lowBound=0, cat='Integer')
ADM_Amsterdam_to_Salinas_BB=pulp.LpVariable('ADM_Amsterdam_to_Salinas_BB', lowBound=0, cat='Integer')
ADM_Amsterdam_to_EU=pulp.LpVariable('ADM_Amsterdam_to_EU', lowBound=0, cat='Integer')
ADM_Amsterdam_to_EEMEA=pulp.LpVariable('ADM_Amsterdam_to_EEMEA', lowBound=0, cat='Integer')
ADM_Amsterdam_to_LA=pulp.LpVariable('ADM_Amsterdam_to_LA', lowBound=0, cat='Integer')
ADM_Amsterdam_to_AP_China=pulp.LpVariable('ADM_Amsterdam_to_AP_China', lowBound=0, cat='Integer')
ADM_Amsterdam_to_China=pulp.LpVariable('ADM_Amsterdam_to_China', lowBound=0, cat='Integer')


In [476]:
x = pulp.LpVariable.dicts("x", range(45), lowBound=0, cat="Integer")

In [477]:
x

{0: x_0,
 1: x_1,
 2: x_2,
 3: x_3,
 4: x_4,
 5: x_5,
 6: x_6,
 7: x_7,
 8: x_8,
 9: x_9,
 10: x_10,
 11: x_11,
 12: x_12,
 13: x_13,
 14: x_14,
 15: x_15,
 16: x_16,
 17: x_17,
 18: x_18,
 19: x_19,
 20: x_20,
 21: x_21,
 22: x_22,
 23: x_23,
 24: x_24,
 25: x_25,
 26: x_26,
 27: x_27,
 28: x_28,
 29: x_29,
 30: x_30,
 31: x_31,
 32: x_32,
 33: x_33,
 34: x_34,
 35: x_35,
 36: x_36,
 37: x_37,
 38: x_38,
 39: x_39,
 40: x_40,
 41: x_41,
 42: x_42,
 43: x_43,
 44: x_44}

In [446]:
np.arange(3)

array([0, 1, 2])

In [443]:
quantity.flatten()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [449]:
cp.Variable(shape=(9,5), quantity)

SyntaxError: positional argument follows keyword argument (<ipython-input-449-b8be25abad6d>, line 1)

In [ ]:
# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A*x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [424]:
result['x']

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [410]:
result['success']

False

In [138]:
prob = LpProblem("costSupplyNetwork", LpMinimize)

In [162]:
suppliers = range(5)
destiny = range(9)

var = LpVariable.dicts("", ((i, j) for i in suppliers for j in destiny),cat='Continuous')

In [163]:
var

{(0, 0): _(0,_0),
 (0, 1): _(0,_1),
 (0, 2): _(0,_2),
 (0, 3): _(0,_3),
 (0, 4): _(0,_4),
 (0, 5): _(0,_5),
 (0, 6): _(0,_6),
 (0, 7): _(0,_7),
 (0, 8): _(0,_8),
 (1, 0): _(1,_0),
 (1, 1): _(1,_1),
 (1, 2): _(1,_2),
 (1, 3): _(1,_3),
 (1, 4): _(1,_4),
 (1, 5): _(1,_5),
 (1, 6): _(1,_6),
 (1, 7): _(1,_7),
 (1, 8): _(1,_8),
 (2, 0): _(2,_0),
 (2, 1): _(2,_1),
 (2, 2): _(2,_2),
 (2, 3): _(2,_3),
 (2, 4): _(2,_4),
 (2, 5): _(2,_5),
 (2, 6): _(2,_6),
 (2, 7): _(2,_7),
 (2, 8): _(2,_8),
 (3, 0): _(3,_0),
 (3, 1): _(3,_1),
 (3, 2): _(3,_2),
 (3, 3): _(3,_3),
 (3, 4): _(3,_4),
 (3, 5): _(3,_5),
 (3, 6): _(3,_6),
 (3, 7): _(3,_7),
 (3, 8): _(3,_8),
 (4, 0): _(4,_0),
 (4, 1): _(4,_1),
 (4, 2): _(4,_2),
 (4, 3): _(4,_3),
 (4, 4): _(4,_4),
 (4, 5): _(4,_5),
 (4, 6): _(4,_6),
 (4, 7): _(4,_7),
 (4, 8): _(4,_8)}

In [142]:
quantity = LpVariable()

In [143]:
quantity

[quantity_3, quantity_4]

In [134]:
results=[]
results.append(bc_supply_penalty(quantity))
results.append(china_tax(calculated_freight_matrix,base_cost_matrix))
results.append(ap_china_tax(calculated_freight_matrix,base_cost_matrix))
results.append(eu_asia_tax(calculated_freight_matrix,base_cost_matrix))

In [135]:
np.sum(results)

4

In [86]:
a = np.array([[1,2,3],[3,4,5]])

In [87]:
a

array([[1, 2, 3],
       [3, 4, 5]])

In [89]:
np.sum(a)

18

In [88]:
a*np.array([2,2,2])

array([[ 2,  4,  6],
       [ 6,  8, 10]])

In [71]:
base_cost

,Location,Cost ($/MT)
0,BC San Pedro,3496
1,BC Malaysia,3942
2,BC Pasir Gudang,3942
3,Olam Singapore,3836
4,Olam Holland,3278


In [79]:
base_cost.values[4,1]*quantity[8,4]

0.0

In [80]:
base_cost.loc[base_cost['Location']=='Olam Holland','Cost ($/MT)'].values[0]*quantity[8,4]

0.0

In [70]:
quantity

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [54]:
quantity[:,:3]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [57]:
a=np.random.randn(3,4)

In [58]:
a

array([[ 0.46860612,  0.64604391,  1.13196826,  0.5475657 ],
       [-1.09338983, -0.05767518, -2.59283151,  1.21904741],
       [ 0.73649776, -0.02234066,  0.32790399, -1.49935875]])

In [66]:
a[2,3]

-1.4993587492172424

In [61]:
np.sum(a[:,:2])

0.6777421175707968

In [51]:
freight_cost(quantity, freight_matrix)

,BC San Pedro,BC Malaysia,BC Pasir Gudang,ADM Singapore,ADM Amsterdam
,,,,,
East Coast BB,0.0,0.0,0.0,0.0,0.0
East Coast SB,0.0,0.0,0.0,0.0,0.0
West Coast BB,0.0,0.0,0.0,0.0,0.0
Salinas BB,0.0,0.0,0.0,0.0,0.0
EU,0.0,0.0,0.0,0.0,0.0
EEMEA,0.0,0.0,0.0,0.0,0.0
LA,0.0,0.0,0.0,0.0,0.0
AP - China,0.0,0.0,0.0,0.0,0.0
China,0.0,0.0,0.0,0.0,0.0
